# Library

In [1]:
import pandas as pd
import numpy as np
import glob
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# TF-IDF

reference: [TF-IDF](https://wikidocs.net/31698)

# Practice

## 기자회견

In [5]:
comments = pd.read_pickle('./data/comments_minheejin.pickle')

In [266]:
kiwi = Kiwi()
# 단어 사전에 단어 추가
kiwi.add_user_word('어도어', 'NNP')
kiwi.add_user_word('빌리프랩', 'NNP')
kiwi.add_user_word('빌리프렙', 'NNP')
kiwi.add_user_word('아일리스', 'NNP')
kiwi.add_user_word('르세라핌', 'NNP')
kiwi.add_user_word('피프티피프티', 'NNP')
kiwi.add_user_word('뉴진스', 'NNP')
kiwi.add_user_word('아일릿', 'NNP')
kiwi.add_user_word('하이브', 'NNP')
kiwi.add_user_word('방시혁', 'NNP')
kiwi.add_user_word('힛뱅맨', 'NNP')
kiwi.add_user_word('힛맨뱅', 'NNP')
kiwi.add_user_word('민희진', 'NNP')
kiwi.add_user_word('미니진', 'NNP')
kiwi.add_user_word('희진', 'NNP')
kiwi.add_user_word('레퍼런스', 'NNG')
kiwi.add_user_word('언플', 'NNG')
kiwi.add_user_word('대퓨', 'NNG')
kiwi.add_user_word('대퓨님', 'NNG')
kiwi.add_user_word('개저씨', 'NNG')

# 불용어 사전에 불용어 추가
stopwords = Stopwords()
stopwords.add(('웩퉥', 'NNG'))
stopwords.add(('결국', 'NNG'))

def extract_tokens(string: str, tokenizer: Kiwi, stopwords: Stopwords, tags={'NNP', 'NNG'}):
    # 주어진 문자열(string)을 입력으로 받아, 지정된 품사 태그와 길이 조건을 만족하는 토큰들을 추출하는 함수

    # Kiwi 객체를 사용하여 문자열을 토크나이즈(tokenize)하고, 불용어(stopwords)를 적용
    tokens = tokenizer.tokenize(string, stopwords=stopwords)
    
    # 토크나이즈된 토큰 중에서, 지정된 태그 집합(tags)에 포함되며 길이가 2 이상인 토큰들의 형태소를 추출하여 리스트에 저장
    target_tokens = [token.form for token in tokens if token.tag in tags and len(token.form) >= 2]

    # 조건을 만족하는 토큰들의 리스트를 반환
    return target_tokens

In [104]:
# comments 데이터프레임의 'textOriginal' 열에 있는 텍스트에 대해
# extract_tokens 함수를 적용하여 추출된 토큰들을 'tokens' 열에 저장
comments['tokens'] = comments.textOriginal.apply(lambda x: extract_tokens(x, kiwi, stopwords))

# 'tokens' 열에 있는 토큰 리스트를 문자열로 변환(join)하여, 
# 각각의 리스트를 공백으로 구분된 문자열로 합치고, 이를 리스트로 변환하여 'tokens' 변수에 저장
tokens = comments.tokens.str.join(' ').tolist()  # 혹은 comments.tokens.apply(lambda x: ' '.join(x))


In [30]:
# CountVectorizer 객체를 생성하여, 단어의 빈도를 계산할 준비를 함
count_vector = CountVectorizer()

# tokens 리스트에 있는 각 문서(토큰 문자열)에 대해, CountVectorizer를 적용하여 문서-단어 행렬(dtm)을 생성
dtm = count_vector.fit_transform(tokens)

# dtm을 배열 형태로 변환한 후, Pandas DataFrame으로 변환
# 각 열은 CountVectorizer에서 추출된 단어(특징)를 나타내고, 각 행은 문서를 나타냄
dtm = pd.DataFrame(
    dtm.toarray(),  # 희소 행렬을 밀집 배열로 변환
    columns=count_vector.get_feature_names_out(),  # 각 열의 이름을 단어(feature)로 설정
)

# 최종적으로 생성된 문서-단어 행렬을 출력
dtm


In [106]:
# TfidfVectorizer 객체를 생성하여, 단어의 TF-IDF 값을 계산할 준비를 함
tfidf = TfidfVectorizer()

# tokens 리스트에 있는 각 문서(토큰 문자열)에 대해, TfidfVectorizer를 적용하여 TF-IDF 행렬(tfidf_vector)을 생성
tfidf_vector = tfidf.fit_transform(tokens)


In [107]:
## for문 사용하지 않고 계산 (optional)

# 유사도를 저장할 리스트 초기화
similarities = []

# tfidf_vector의 각 문서 벡터에 대해 반복문 실행 (진행 상황을 표시하기 위해 tqdm 사용)
for vector in tqdm(tfidf_vector.toarray()):
    # 첫 번째 문서를 기준(reference)으로 설정
    reference = tfidf_vector.toarray()[0]
    
    # 코사인 유사도 계산: 두 벡터의 내적을 벡터의 크기(노름)의 곱으로 나눈 값
    cos_sim = vector @ reference.T / (np.sqrt((vector**2).sum()) * np.sqrt((reference**2).sum()))
    
    # 계산된 코사인 유사도를 similarities 리스트에 추가
    similarities.append(cos_sim)


  0%|          | 24/10169 [00:01<07:36, 22.24it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_15420\2504669301.py:7: RuntimeWarning: invalid value encountered in scalar divide
  cos_sim = vector@reference.T / (np.sqrt((vector**2).sum()) * np.sqrt((reference**2).sum()))
 10%|█         | 1041/10169 [00:47<06:57, 21.87it/s]


KeyboardInterrupt: 

In [141]:
# tfidf_vector의 문서들 간의 코사인 유사도를 계산
similarities = cosine_similarity(tfidf_vector.toarray())

# 대각선의 값(자기 자신과의 유사도)을 0으로 채워서 제외
np.fill_diagonal(similarities, 0)

# 유사도가 0.9를 초과하는 위치(인덱스)를 반환
np.where(similarities > 0.9)


(array([    4,     4,     6, ..., 10162, 10163, 10168], dtype=int64),
 array([ 3040, 10068,  2750, ...,  7847,  4162, 10142], dtype=int64))

## 수정이

In [223]:
# day 39 comments data, video_id: JPaubSOSxeM에 대해서 유사 댓글 분석
comments = pd.read_csv('./data/comments.csv')
comments = comments.query('video_id=="JPaubSOSxeM"').drop(columns=['video_id'])
comments['tokens'] = comments.comments.apply(lambda x: extract_tokens(x, kiwi, stopwords))

In [224]:
tf_idf = TfidfVectorizer()
tf_idf_vector = tf_idf.fit_transform(comments.tokens.str.join(' ').tolist())

In [225]:
tf_idf_vector = tf_idf_vector.toarray()
similarities = cosine_similarity(tf_idf_vector)
np.fill_diagonal(similarities, 0)

In [213]:
np.where(similarities > 0.7)

(array([   1,    4,    5, ..., 3836, 3836, 3836], dtype=int64),
 array([  28, 2062,  589, ..., 3688, 3702, 3729], dtype=int64))

In [215]:
comments.iloc[[5, 589]]

,comments,tokens
5,아영님 짧대에서 보다가 여기서 보니까 신기하당,"[아영, 짧대]"
589,아영님 어디서 많이 봤다했더니 짧대에 나오신분이셨네요,[짧대]


## 뉴스

In [250]:
news = []
file_paths = glob.glob('./data/news_data/**/*.txt', recursive=True)
for file_path in tqdm(file_paths):
    with open(file_path) as file:
        temp = file.read()
        news.append(temp)

100%|██████████| 1600/1600 [00:00<00:00, 14737.48it/s]


In [291]:
news = pd.DataFrame(news, columns=['contents'])
news = news.drop_duplicates()
news.contents = news.contents.str.replace('\s{1,}', ' ', regex=True)
news['tokens'] = news.contents.apply(lambda x: extract_tokens(x, kiwi, stopwords))

In [292]:
tokens = news.tokens.str.join(' ').tolist()
tf_idf = TfidfVectorizer()
tf_idf_vector = tf_idf.fit_transform(tokens)
cos_sim = cosine_similarity(tf_idf_vector.toarray())
np.fill_diagonal(cos_sim, 0)

In [293]:
(np.where(cos_sim > 0.9)[0][:5], np.where(cos_sim > 0.9)[1][:5])

(array([195, 232, 239, 242, 289], dtype=int64),
 array([ 289,  239,  232, 1025,  195], dtype=int64))

In [296]:
news.iloc[[242, 1025]]

,contents,tokens
248,"반포현대 재건축 부담금 850만→1억4천만원으로 16배 '껑충'(종합) 서초구청, ...","[반포, 현대, 건축, 부담금, 종합, 서초구청, 조합, 통지, 조합, 구청, 고무..."
1049,"재건축 부담금 '첫타자' 반포현대, 예상액 1억4천만원 서초구청, 조합에 통지…조합...","[건축, 부담금, 타자, 반포, 현대, 예상액, 서초구청, 조합, 통지, 조합, 예..."


## 다트 사업의 개요

In [16]:
data = pd.read_pickle('./data/crawl_data.pickle')
data = data.query('dates == "2022.03"')

In [17]:
data['business_info'] = data['사업의 개요'].apply(lambda x: BeautifulSoup(x, 'lxml').text)
data.business_info = data.business_info.str.replace('\s+', ' ', regex=True)
data.business_info = data.business_info.str.replace('\d?\.? ? 사업의 개요 ?', '', regex=True)

In [20]:
def extract_tokens(string: str, tokenizer: Kiwi, stopwords: Stopwords, tags={'NNP', 'NNG'}):
    # 주어진 문자열(string)을 입력으로 받아, 지정된 품사 태그와 길이 조건을 만족하는 토큰들을 추출하는 함수

    # Kiwi 객체를 사용하여 문자열을 토크나이즈(tokenize)하고, 불용어(stopwords)를 적용
    tokens = tokenizer.tokenize(string, stopwords=stopwords)
    
    # 토크나이즈된 토큰 중에서, 지정된 태그 집합(tags)에 포함되며 길이가 2 이상인 토큰들의 형태소를 추출하여 리스트에 저장
    target_tokens = [token.form for token in tokens if token.tag in tags and len(token.form) >= 2]

    # 조건을 만족하는 토큰들의 리스트를 반환
    return target_tokens

kiwi = Kiwi()
kiwi.add_user_word('바이오시밀러', 'NNG')
kiwi.add_user_word('웅진씽크빅', 'NNP')

stopwords = Stopwords()
stopwords.add(('당사', 'NNG'))
stopwords.add(('산업', 'NNG'))
stopwords.add(('특성', 'NNG'))

data['tokens'] = data.business_info.apply(lambda x: extract_tokens(x, kiwi, stopwords))
data['tokens_joined'] = data.tokens.str.join(' ')

In [26]:
documents = data.tokens_joined.tolist()
tf_idf = TfidfVectorizer()
tf_idf_vector = tf_idf.fit_transform(documents)
similarities = cosine_similarity(tf_idf_vector.toarray())
np.fill_diagonal(similarities, 0)

In [45]:
np.where(similarities > 0.4)

(array([  0,   0,   1, ..., 789, 789, 790], dtype=int64),
 array([346, 775, 439, ..., 326, 741, 608], dtype=int64))

In [49]:
data.iloc[[0, 446]]

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm,modified_report_nm,dates,회사의 개요,사업의 개요,business_info,tokens,tokens_joined
0,01510489,프레스티지바이오파마,950210,Y,분기보고서 (2022.03),20220530001024,프레스티지바이오파마,20220530,,분기보고서 (2022.03),2022.03,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...","<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",당사는 바이오시밀러와 항체 신약 개발에 집중해 온 항체의약품 개발 전문 제약회사로...,"[바이오시밀러, 항체, 신약, 개발, 집중, 항체, 의약품, 개발, 전문, 제약, ...",바이오시밀러 항체 신약 개발 집중 항체 의약품 개발 전문 제약 회사 일자 한국 유가...
446,00401731,LG전자,066570,Y,분기보고서 (2022.03),20220516001009,LG전자,20220516,,분기보고서 (2022.03),2022.03,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...","<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",연결회사의 보고부문인 사업본부는 서로 다른 제품과 용역을 제공하는 전략적 사업단위...,"[연결, 회사, 보고, 부문, 사업, 본부, 제품, 용역, 제공, 전략, 사업, 단...",연결 회사 보고 부문 사업 본부 제품 용역 제공 전략 사업 단위 사업 기술 마케팅 ...
